<a href="https://colab.research.google.com/github/raz0208/Advance-Quantitative-Research-Project/blob/main/Task2/Natural_Gas_Storage_Contract_Pricing_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Natural Gas Storage Contract Pricing Prototype**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# DATA LOADING & PROCESSING
try:
    Nat_Gas_df = pd.read_csv('Nat_Gas.csv')
    ETS_df = pd.read_csv('ETS_Nat_Gas_Forcasted_Prices(Monthly).csv')

    # Preprocessing
    Nat_Gas_df['Dates'] = pd.to_datetime(Nat_Gas_df['Dates'])
    Nat_Gas_df = Nat_Gas_df.set_index('Dates')
    ETS_df['Dates'] = pd.to_datetime(ETS_df['Dates'])
    ETS_df = ETS_df.set_index('Dates')

    # Interpolation
    hist_daily_index = pd.date_range(start=Nat_Gas_df.index.min(), end=Nat_Gas_df.index.max(), freq='D')
    hist_daily_curve = Nat_Gas_df.reindex(hist_daily_index).interpolate(method='linear')
    hist_daily_curve.index.name = 'Dates'

    forecast_daily_index = pd.date_range(start=ETS_df.index.min(), end=ETS_df.index.max(), freq='D')
    forecast_daily_curve = ETS_df.reindex(forecast_daily_index).interpolate(method='linear')
    forecast_daily_curve.index.name = 'Dates'

    daily_forward_curve = pd.concat([hist_daily_curve, forecast_daily_curve])
    daily_forward_curve = daily_forward_curve[~daily_forward_curve.index.duplicated(keep='last')]
    daily_forward_curve.columns = ['Prices']

    print("Data Loaded Successfully.")

except Exception as e:
    print(f"Error loading data: {e}")
    # Create dummy data if files fail so UI still renders
    dates = pd.date_range('2024-01-01', '2025-12-31', freq='D')
    daily_forward_curve = pd.DataFrame({'Prices': 10 + np.sin(np.linspace(0, 10, len(dates)))}, index=dates)

# PRICING FUNCTION
def calculate_contract_value(inj_sched, with_sched, max_vol, store_cost, inj_cost, trans_cost):
    events = []
    # Parse Injection
    for line in inj_sched.split('\n'):
        if ',' in line:
            d, v = line.split(',')
            events.append({'Date': pd.to_datetime(d.strip()), 'Type': 'Injection', 'Requested_Volume': v.strip()})

    # Parse Withdrawal
    for line in with_sched.split('\n'):
        if ',' in line:
            d, v = line.split(',')
            events.append({'Date': pd.to_datetime(d.strip()), 'Type': 'Withdrawal', 'Requested_Volume': v.strip()})

    if not events: return 0, pd.DataFrame()

    schedule = pd.DataFrame(events).sort_values('Date').reset_index(drop=True)
    inventory = 0
    results = []
    price_data = daily_forward_curve.sort_index()

    for i, row in schedule.iterrows():
        curr_date = row['Date']
        action = row['Type']
        req_vol = row['Requested_Volume']

        # Get Price
        try:
            if curr_date in price_data.index: price = price_data.loc[curr_date, 'Prices']
            else: price = price_data.asof(curr_date)['Prices']
        except: price = 0

        # Calculate
        vol = 0
        cash = 0

        if action == 'Injection':
            try: v_float = float(req_vol)
            except: v_float = 0
            vol = min(v_float, max_vol - inventory)
            vol = max(0, vol)

            cost = (vol * price) + (vol * inj_cost) + trans_cost
            cash = -cost
            inventory += vol

        elif action == 'Withdrawal':
            if str(req_vol).lower() == 'all':
                vol = inventory
            else:
                try: v_float = float(req_vol)
                except: v_float = 0
                vol = min(v_float, inventory)
                vol = max(0, vol)

            rev = (vol * price)
            cost = (vol * inj_cost) + trans_cost
            cash = rev - cost
            inventory -= vol

        results.append({
            'Date': curr_date, 'Action': action, 'Price': price,
            'Requested_Volume': req_vol, 'Executed_Volume': vol,
            'Inventory': inventory, 'Cash_Flow': cash
        })

    df = pd.DataFrame(results)

    total_store_cost = 0
    if not df.empty:
        duration = (df['Date'].max() - df['Date'].min()).days / 30.44
        duration = max(1, duration)
        total_store_cost = duration * store_cost

    final_val = df['Cash_Flow'].sum() - total_store_cost
    return final_val, df

# WIDGETS GUI

# Inputs
style = {'description_width': 'initial'}
w_max_vol = widgets.FloatText(value=2000000, description='Max Volume:', style=style)
w_store_cost = widgets.FloatText(value=100000, description='Storage Cost/Month:', style=style)
w_inj_cost = widgets.FloatText(value=0.01, description='Inj/With Cost per Unit:', style=style)
w_trans_cost = widgets.FloatText(value=50000, description='Transport Cost/Trip:', style=style)

w_inj_sched = widgets.Textarea(
    value="2024-10-31, 1000000\n2024-11-15, 80000",
    description='Injection Schedule:',
    placeholder='YYYY-MM-DD, Volume',
    layout=widgets.Layout(height='100px', width='90%'), style=style
)

w_with_sched = widgets.Textarea(
    value="2025-01-15, 90000\n2025-02-15, All",
    description='Withdrawal Schedule:',
    placeholder='YYYY-MM-DD, Volume',
    layout=widgets.Layout(height='100px', width='90%'), style=style
)

btn_calc = widgets.Button(description="Calculate Value", button_style='success')
output = widgets.Output()

def on_click_calc(b):
    with output:
        clear_output()
        val, df = calculate_contract_value(
            w_inj_sched.value, w_with_sched.value,
            w_max_vol.value, w_store_cost.value, w_inj_cost.value, w_trans_cost.value
        )

        # Display Value
        color = "green" if val >= 0 else "red"
        display(widgets.HTML(f"<h2>Contract Value: <span style='color:{color}'>${val:,.2f}</span></h2>"))

        if not df.empty:
            # Display Table
            display(df[['Date', 'Action', 'Price', 'Executed_Volume', 'Inventory', 'Cash_Flow']])

            # Display Plots
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

            # Price & Actions
            start = df['Date'].min() - pd.Timedelta(days=30)
            end = df['Date'].max() + pd.Timedelta(days=30)
            mask = (daily_forward_curve.index >= start) & (daily_forward_curve.index <= end)
            subset = daily_forward_curve.loc[mask]

            ax1.plot(subset.index, subset['Prices'], color='lightgray', label='Market Price')
            inj = df[df['Action'] == 'Injection']
            withd = df[df['Action'] == 'Withdrawal']
            ax1.scatter(inj['Date'], inj['Price'], color='green', s=80, label='Buy', zorder=5)
            ax1.scatter(withd['Date'], withd['Price'], color='red', s=80, label='Sell', zorder=5)
            ax1.set_title("Price Curve & Actions")
            ax1.legend()
            ax1.grid(True, alpha=0.3)

            # Inventory
            ax2.plot(df['Date'], df['Inventory'], marker='o', linestyle='-', color='blue')
            ax2.fill_between(df['Date'], df['Inventory'], color='blue', alpha=0.1)
            ax2.set_title("Inventory Level")
            ax2.grid(True, alpha=0.3)

            plt.tight_layout()
            plt.show()

            # CashFlow Waterfall/Bar
            fig, ax = plt.subplots(figsize=(10, 6))
            colors = ['green' if x >= 0 else 'red' for x in df['Cash_Flow']]
            ax.bar(df['Date'], df['Cash_Flow'], width=5, color=colors)
            ax.axhline(0, color='black', linewidth=0.8)
            ax.set_title("Cash Flow per Event")
            ax.grid(True, axis='y', alpha=0.3)
            plt.show()

btn_calc.on_click(on_click_calc)

# Layout
ui = widgets.VBox([
    widgets.HTML("<h3>Gas Storage Pricing Model</h3>"),
    widgets.HBox([widgets.VBox([w_max_vol, w_store_cost]), widgets.VBox([w_inj_cost, w_trans_cost])]),
    widgets.HBox([w_inj_sched, w_with_sched]),
    btn_calc,
    output
])

display(ui)